In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pickle

In [3]:
dataset = pickle.load(open('./dataset/total_dataset_condition_pred','rb'))

In [4]:
dataset
dataset.drop(['pet_id','listing_date','issue_date'],axis=1,inplace=True)

In [5]:
train = dataset.iloc[:18834,:] 
test = dataset.iloc[18834:,:] 

In [6]:
train.isnull().sum()

X1                0
X2                0
breed_category    0
condition         0
height(cm)        0
                 ..
Tortie Point      0
Tricolor          0
White             0
Yellow            0
Yellow Brindle    0
Length: 62, dtype: int64

In [7]:
test.drop(['pet_category','breed_category'],axis=1,inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
test.shape,train.shape

((8072, 60), (18834, 62))

In [9]:
X_train = train.drop(['pet_category','breed_category'],axis=1)
y_train_breed = train['breed_category']
y_train_pet = train['pet_category']

In [10]:
X_train

,X1,X2,condition,height(cm),length(m),Apricot,Black,Black Brindle,Black Smoke,Black Tabby,...,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle
0,13,9,2.0,7.78,0.80,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,13,9,1.0,14.19,0.72,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,15,4,2.0,40.90,0.15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1.0,17.82,0.62,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,18,4,2.0,11.06,0.50,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18829,0,1,2.0,27.36,0.44,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
18830,15,4,2.0,14.25,0.73,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18831,13,9,0.0,28.13,0.99,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18832,13,9,0.0,44.82,0.55,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [11]:
X_train = np.array(X_train)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,:4] = sc.fit_transform(X_train[:, :4])

In [13]:
y_train_breed = np.array(y_train_breed)

# lets a model for breed

In [14]:
from sklearn.model_selection import train_test_split
X_train_breed,X_test_breed,y_train_breed,y_test_breed = train_test_split(X_train,y_train_breed,test_size = 0.2, random_state = 0)

In [26]:
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.svm import SVC
svc_classifier = SVC(kernel='linear',random_state=0)
svc_classifier.fit(X_train_breed,y_train_breed)
y_pred = svc_classifier.predict(X_test_breed)
print(confusion_matrix(y_test_breed,y_pred))
print("SVM linear classifier accuracy : "+str(accuracy_score(y_test_breed,y_pred)))

[[1736    3   38]
 [ 335 1317   24]
 [ 154   16  144]]
SVM linear classifier accuracy : 0.8486859569949562


In [27]:
svc_kernel =  SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,decision_function_shape='ovr', degree=3, gamma=0.1, kernel='poly',max_iter=-1, probability=False, random_state=None, shrinking=True,tol=0.001, verbose=False)
svc_kernel.fit(X_train_breed,y_train_breed)
y_pred = svc_kernel.predict(X_test_breed)
print(confusion_matrix(y_test_breed,y_pred))
print("SVM polly kernel classifier accuracy : "+str(accuracy_score(y_test_breed,y_pred)))

[[1740   10   27]
 [ 334 1320   22]
 [ 157   17  140]]
SVM polly kernel classifier accuracy : 0.8494823466949828


In [29]:
from sklearn.naive_bayes import GaussianNB
Gnb = GaussianNB()
Gnb.fit(X_train_breed,y_train_breed)
y_pred = Gnb.predict(X_test_breed)
print(confusion_matrix(y_test_breed,y_pred))
print("GaussianNB classifier accuracy : "+str(accuracy_score(y_test_breed,y_pred)))

[[ 202    8 1567]
 [ 188    7 1481]
 [  19    1  294]]
GaussianNB classifier accuracy : 0.1335280063711176


In [33]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
tree_para = {'criterion':['gini','entropy'],'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}
Grid = GridSearchCV(DecisionTreeClassifier(),tree_para,cv=5)
Grid.fit(X_train_breed,y_train_breed)

print(Grid.best_params_) 
print(Grid.best_estimator_) 
print(Grid.best_score_)

{'criterion': 'entropy', 'max_depth': 7}
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
0.8580341142895068


In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
Grid = GridSearchCV(RandomForestClassifier(),param_grid,cv=5)
Grid.fit(X_train_breed,y_train_breed)


print(Grid.best_params_) 
print(Grid.best_estimator_) 
print(Grid.best_score_)

{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 500}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=8, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
0.855113824915378


In [ ]:
from sklearn.model_selection import GridSearchCV 
  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf','poly','sigmoid']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid.fit(X_train_breed, y_train_breed)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 75 candidates, totalling 225 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.839, total=  18.1s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.0s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.853, total=  18.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   36.0s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.855, total=  18.1s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.848, total=  13.2s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.858, total=  12.8s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.860, total=  12.4s
[CV] C=0.1, gamma=1, kernel=sigmoid ..................................
[CV] ...... C=0.1, gamma=1, kernel=sigmoid, score=0.493, total=  24.3s
[CV] C=0.1, gamma=1, kernel=sigmoid ..................................
[CV] ...... C=0.1, gamma=1, kernel=sigmoid, score=0.479, total=  25.1s
[CV] C=0.1, gamma=1, kernel=sigmoid ..................................
[CV] ...... C=0.1, gamma=1, kernel=sigmoid, score=0.474, total=  25.3s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] .

[CV] ...... C=1, gamma=0.1, kernel=sigmoid, score=0.663, total=   8.0s
[CV] C=1, gamma=0.1, kernel=sigmoid ..................................
[CV] ...... C=1, gamma=0.1, kernel=sigmoid, score=0.659, total=   9.5s
[CV] C=1, gamma=0.1, kernel=sigmoid ..................................
[CV] ...... C=1, gamma=0.1, kernel=sigmoid, score=0.643, total=   9.7s
[CV] C=1, gamma=0.01, kernel=rbf .....................................
[CV] ......... C=1, gamma=0.01, kernel=rbf, score=0.840, total=  11.0s
[CV] C=1, gamma=0.01, kernel=rbf .....................................
[CV] ......... C=1, gamma=0.01, kernel=rbf, score=0.851, total=   9.3s
[CV] C=1, gamma=0.01, kernel=rbf .....................................
[CV] ......... C=1, gamma=0.01, kernel=rbf, score=0.841, total=   9.4s
[CV] C=1, gamma=0.01, kernel=poly ....................................
[CV] ........ C=1, gamma=0.01, kernel=poly, score=0.479, total=  12.1s
[CV] C=1, gamma=0.01, kernel=poly ....................................
[CV] .

[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.844, total=   8.7s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.838, total=   9.4s
[CV] C=10, gamma=0.001, kernel=poly ..................................
[CV] ...... C=10, gamma=0.001, kernel=poly, score=0.479, total=   9.5s
[CV] C=10, gamma=0.001, kernel=poly ..................................
[CV] ...... C=10, gamma=0.001, kernel=poly, score=0.479, total=   9.7s
[CV] C=10, gamma=0.001, kernel=poly ..................................
[CV] ...... C=10, gamma=0.001, kernel=poly, score=0.479, total=   9.5s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.835, total=   8.7s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.844, total=  10.1s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] .

[CV] .... C=100, gamma=0.0001, kernel=poly, score=0.479, total=   9.2s
[CV] C=100, gamma=0.0001, kernel=sigmoid .............................
[CV] . C=100, gamma=0.0001, kernel=sigmoid, score=0.835, total=   8.6s
[CV] C=100, gamma=0.0001, kernel=sigmoid .............................
[CV] . C=100, gamma=0.0001, kernel=sigmoid, score=0.844, total=   8.2s
[CV] C=100, gamma=0.0001, kernel=sigmoid .............................
[CV] . C=100, gamma=0.0001, kernel=sigmoid, score=0.838, total=   9.3s
[CV] C=1000, gamma=1, kernel=rbf .....................................
[CV] ......... C=1000, gamma=1, kernel=rbf, score=0.834, total=  21.2s
[CV] C=1000, gamma=1, kernel=rbf .....................................
[CV] ......... C=1000, gamma=1, kernel=rbf, score=0.841, total=  35.9s
[CV] C=1000, gamma=1, kernel=rbf .....................................
[CV] ......... C=1000, gamma=1, kernel=rbf, score=0.845, total=  36.7s
[CV] C=1000, gamma=1, kernel=poly ....................................


In [ ]:
print(grid.best_params_) 
print(grid.best_estimator_)

In [16]:
from sklearn.svm import SVC
svc = SVC(C=100, gamma=0.01, kernel='rbf')
svc.fit(X_train_breed,y_train_breed)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [21]:
from sklearn.metrics import confusion_matrix,accuracy_score
y_pred = svc.predict(X_test_breed)
print(confusion_matrix(y_test_breed,y_pred))
print("SVC kernel rbf classifier accuracy : "+str(accuracy_score(y_test_breed,y_pred)))

[[1674   67   36]
 [ 273 1378   25]
 [ 125   36  153]]
SVC kernel rbf classifier accuracy : 0.8508096628616937


In [18]:
final_model_breed = SVC(C=100, gamma=0.01, kernel='rbf')
final_model_breed.fit(X_train_breed,y_train_breed)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# lets train pet

In [19]:
from sklearn.model_selection import train_test_split
X_train_pet,X_test_pet,y_train_pet,y_test_pet = train_test_split(X_train,y_train_pet,test_size = 0.2, random_state = 0)

In [22]:
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.svm import SVC
svc_classifier = SVC(kernel='linear',random_state=0)
svc_classifier.fit(X_train_pet,y_train_pet)
y_pred = svc_classifier.predict(X_test_pet)
print(confusion_matrix(y_test_pet,y_pred))
print("SVM linear classifier accuracy : "+str(accuracy_score(y_test_pet,y_pred)))

[[   1    3    8    1]
 [   0 1156  297    6]
 [   1  165 1935   13]
 [   0   36   39  106]]
SVM linear classifier accuracy : 0.8489514202282984


In [23]:
from sklearn.svm import SVC
svc = SVC(C=100, gamma=0.01, kernel='rbf')
svc.fit(X_train_pet,y_train_pet)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [24]:
from sklearn.metrics import confusion_matrix,accuracy_score
y_pred = svc.predict(X_test_pet)
print(confusion_matrix(y_test_pet,y_pred))
print("SVC kernel rbf classifier accuracy : "+str(accuracy_score(y_test_pet,y_pred)))

[[   1    2    8    2]
 [   0 1143  298   18]
 [   0  152 1947   15]
 [   0   15   36  130]]
SVC kernel rbf classifier accuracy : 0.8550570745951686


In [27]:
test  = np.array(test)

In [28]:
test[:,:4] = sc.transform(test[:, :4])

In [29]:
test_pet = svc.predict(test)

In [30]:
test_breed = final_model_breed.predict(test)

In [31]:
testDataset = pd.read_csv("./dataset/test.csv")
testDataset.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [32]:
sub_df = pd.DataFrame({'pet_id': testDataset['pet_id'], 'breed_category': test_breed,'pet_category':test_pet})
sub_df.to_csv('submit_new.csv', index=False)